## *Install Libraries*

In [ ]:
!pip install tensorflow==2.15.0
!pip install mne==1.6.0
!pip install braindecode===0.7
!pip install -q -U keras-tuner
!pip install git+https://github.com/Anlopezbo/EEGVAENL.git
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases

## *Import Libraries*

In [ ]:
from scipy.signal import resample
from scipy.signal import freqz, filtfilt, resample
from scipy.signal import butter as bw
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from braindecode.datasets.moabb import MOABBDataset
from gcpds.databases import GIGA_MI_ME
from braindecode.preprocessing.preprocess import (exponential_moving_standardize, preprocess, Preprocessor, scale)
from braindecode.preprocessing.windowers import create_windows_from_events
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,Callback, CSVLogger, ReduceLROnPlateau
import time
import csv
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input, Flatten, Lambda
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.losses import CategoricalCrossentropy, binary_crossentropy, Loss
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout, Add, Lambda, Permute
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization, Reshape
from tensorflow.keras.layers import SpatialDropout2D, UpSampling2D
from keras_tuner import HyperParameters, GridSearch
from keras_tuner.tuners import RandomSearch, BayesianOptimization
from keras_tuner import RandomSearch
from keras_tuner import Objective
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Layer
from tensorflow.python.keras.utils import losses_utils
from keras_tuner import BayesianOptimization as OriginalBayesianOptimization
from keras_tuner.oracles import BayesianOptimizationOracle as OriginalBayesianOptimizationOracle
import sklearn
import sklearn.exceptions
import sklearn.gaussian_process
from keras_tuner.src.api_export import keras_tuner_export
from keras_tuner.src.engine import hyperparameters as hp_module
from keras_tuner.src.engine import oracle as oracle_module
from keras_tuner.src.engine import trial as trial_module
from keras_tuner.src.engine import tuner as tuner_module

## *Load Data*

In [ ]:
channels = ['Fp1','Fpz','Fp2',
            'AF7','AF3','AFz','AF4','AF8',
            'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
            'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
            'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
            'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
            'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
            'PO7','PO3','POz','PO4','PO8',
            'O1','Oz','O2',
            'Iz']

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    #'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}

arcs = [
    #'hemispheres',
    'areas',
    'channels',
]

In [ ]:
db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
load_args = dict(db = db,
                 eeg_ch_names = channels,
                 fs = db.metadata['sampling_rate'],
                 f_bank = np.asarray([[4., 40.]]),
                 vwt = np.asarray([[2.5, 5]]),
                 new_fs = 128.)

In [ ]:
def butterworth_digital_filter(X, N, Wn, btype, fs, axis=-1, padtype=None, padlen=0, method='pad', irlen=None):
  """
  Apply digital butterworth filter
  INPUT
  ------
  1. X: (D array)
    array with signals.
  2. N: (int+)
    The order of the filter.
  3. Wn: (float+ or 1D array)
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 vector.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”).
    If fs is not specified, Wn units are normalized from 0 to 1, where 1 is the Nyquist frequency (Wn is thus in half cycles / sample and defined as 2*critical frequencies / fs). If fs is specified, Wn is in the same units as fs.
  4. btype: (str) {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}
    The type of filter
  5. fs: (float+)
    The sampling frequency of the digital system.
  6. axis: (int), Default=1.
    The axis of x to which the filter is applied.
  7. padtype: (str) or None, {'odd', 'even', 'constant'}
    This determines the type of extension to use for the padded signal to which the filter is applied. If padtype is None, no padding is used. The default is ‘odd’.
  8. padlen: (int+) or None, Default=0
    The number of elements by which to extend x at both ends of axis before applying the filter. This value must be less than x.shape[axis] - 1. padlen=0 implies no padding.
  9. method: (str), {'pad', 'gust'}
    Determines the method for handling the edges of the signal, either “pad” or “gust”. When method is “pad”, the signal is padded; the type of padding is determined by padtype
    and padlen, and irlen is ignored. When method is “gust”, Gustafsson’s method is used, and padtype and padlen are ignored.
  10. irlen: (int) or None, Default=nONE
    When method is “gust”, irlen specifies the length of the impulse response of the filter. If irlen is None, no part of the impulse response is ignored.
    For a long signal, specifying irlen can significantly improve the performance of the filter.
  OUTPUT
  ------
  X_fil: (D array)
    array with filtered signals.
  """
  b, a = bw(N, Wn, btype, analog=False, output='ba', fs=fs)
  return filtfilt(b, a, X, axis=axis, padtype=padtype, padlen=padlen, method=method, irlen=irlen)

class TimeFrequencyRpr(BaseEstimator, TransformerMixin):
  """
  Time frequency representation of EEG signals.

  Parameters
  ----------
    1. sfreq:  (float) Sampling frequency in Hz.
    2. f_bank: (2D array) Filter banks Frequencies. Default=None
    3. vwt:    (2D array) Interest time windows. Default=None
  Methods
  -------
    1. fit(X, y=None)
    2. transform(X, y=None)
  """
  def __init__(self, sfreq, f_bank=None, vwt=None):
    self.sfreq = sfreq
    self.f_bank = f_bank
    self.vwt = vwt
# ------------------------------------------------------------------------------

  def _validation_param(self):
    """
    Validate Time-Frequency characterization parameters.
    INPUT
    -----
      1. self
    ------
      2. None
    """
    if self.sfreq <= 0:
      raise ValueError('Non negative sampling frequency is accepted')


    if self.f_bank is None:
      self.flag_f_bank = False
    elif self.f_bank.ndim != 2:
      raise ValueError('Band frequencies have to be a 2D array')
    else:
      self.flag_f_bank = True

    if self.vwt is None:
      self.flag_vwt = False
    elif self.vwt.ndim != 2:
      raise ValueError('Time windows have to be a 2D array')
    else:
      self.flag_vwt = True

# ------------------------------------------------------------------------------
  def _filter_bank(self, X):
    """
    Filter bank Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_f: (4D array) set of filtered EEG signals, shape (trials, channels, time_samples, frequency_bands)
    """
    X_f = np.zeros((X.shape[0], X.shape[1], X.shape[2], self.f_bank.shape[0])) #epochs, Ch, Time, bands
    for f in np.arange(self.f_bank.shape[0]):
      X_f[:,:,:,f] = butterworth_digital_filter(X, N=5, Wn=self.f_bank[f], btype='bandpass', fs=self.sfreq)
    return X_f

# ------------------------------------------------------------------------------
  def _sliding_windows(self, X):
    """
    Sliding Windows Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_w: (4D array) shape (trials, channels, window_time_samples, number_of_windows)
    """
    window_lenght = int(self.sfreq*self.vwt[0,1] - self.sfreq*self.vwt[0,0])
    X_w = np.zeros((X.shape[0], X.shape[1], window_lenght, self.vwt.shape[0]))
    for w in np.arange(self.vwt.shape[0]):
        X_w[:,:,:,w] = X[:,:,int(self.sfreq*self.vwt[w,0]):int(self.sfreq*self.vwt[w,1])]
    return X_w

# ------------------------------------------------------------------------------
  def fit(self, X, y=None):
    """
    fit.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
      2. y: (1D array) target labels. Default=None
    OUTPUT
    ------
      1. None
    """
    pass

# ------------------------------------------------------------------------------
  def transform(self, X, y=None):
    """
    Time frequency representation of EEG signals.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, times)
    OUTPUT
    ------
      1. X_wf: (5D array) Time-frequency representation of EEG signals, shape (trials, channels, window_time_samples, number_of_windows, frequency_bands)
    """
    self._validation_param()     #Validate sfreq, f_freq, vwt

    #Avoid edge effects of digital filter, 1st:fbk, 2th:vwt
    if self.flag_f_bank:
        X_f = self._filter_bank(X)
    else:
        X_f = X[:,:,:,np.newaxis]

    if self.flag_vwt:
      X_wf = []
      for f in range(X_f.shape[3]):
        X_wf.append(self._sliding_windows(X_f[:,:,:,f]))
      X_wf = np.stack(X_wf, axis=-1)
    else:
      X_wf = X_f[:,:,:,np.newaxis,:]

    return X_wf

In [ ]:
def load_GIGA(db,
              sbj,
              eeg_ch_names,
              fs,
              f_bank,
              vwt,
              new_fs,
              run=None):

    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    if run == None:
        X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    else:
        X, y = db.get_run(run, classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:, index_eeg_chs, :] #spatial rearrangement
    X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        pass#print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

    return X, y

## *NORMALIZED LOSSES FUNCTIONS FOR DEEP LEARNING:*

 Dado un conjunto de datos de $ K-clases $ con etiquetas ruidosas, denotado como $ D = \{(x, y^{(i)})\}_{i=1}^n $, donde $ x \in X \subset \mathbb{R}^d $ representa una muestra, $ y \in Y = \{1, \ldots, K\}$ es su etiqueta anotada (que puede ser incorrecta), consideramos la distribución sobre diferentes etiquetas para cada muestra $ x{\text{}}$, denotada por $ q(k|x) $. Se cumple que $ \sum_{k=1}^K q(k|x) = 1 $.

 En este documento, nos enfocamos en el caso común donde hay una única etiqueta $ y{\text{}} $ para $ x{\text{}} $: es decir, $ q(y|x) = 1 $ y $ q(k \neq y|x) = 0 $. En este caso, $ q $ es simplemente la codificación one-hot de la etiqueta.


#### *NORMALIZED LOSS*
$$
L_{\text{norm}} = \frac{L(f(x), y)}{\sum_{j=1}^K L(f(x), j)}
$$

#### *CROSS ENTROPY:*

$$
CE = {-\sum_{k=1}^{K}\lg{q(k\mid X)}*\lg{P(k\mid X)}}
$$

#### *NORMALIZED CROSS ENTROPY:*

$$
NCE = \frac{-\sum_{k=1}^{K}\lg{q(k\mid X)}*\lg{P(k\mid X)}}{-\sum_{j=1}^{K}\sum_{k=0}^{K}\lg{q(y=j\mid X)}*\lg{P(k\mid X)}}
$$

Denotamos la etiqueta verdadera de $x$ como $y^*$. Aunque las etiquetas ruidosas pueden surgir de diferentes maneras, una suposición común es que, dado las etiquetas verdaderas, el ruido es condicionalmente independiente de las entradas, es decir, $q(y = k \mid y^* = j, x) = q(y = k \mid y^* = j)$.

Bajo esta suposición, el ruido en las etiquetas puede ser simétrico (o uniforme) o asimétrico (o condicional a la clase). Denotamos la tasa general de ruido por $\eta \in [0, 1]$ y la tasa de ruido específica de clase de $j$ a $k$ por $\eta_{jk}$. Entonces, para ruido simétrico, $\eta_{jk} = \frac{\eta}{K-1}$ para $j \neq k$ y $\eta_{jk} = 1 - \eta$ para $j = k$.

Para ruido asimétrico, $\eta_{jk}$ está condicionado tanto a la clase verdadera $j$ como a la clase mal etiquetada $k$.

La clasificación consiste en aprender una función $f : X \to Y$ que mapea el espacio de entrada al espacio de etiquetas.

**Lema 1.** En un problema de clasificación multiclase, cualquier función de pérdida normalizada $L_{\text{norm}}$ es tolerante al ruido bajo ruido de etiqueta simétrico (o uniforme), si la tasa de ruido $\eta < \frac{K-1}{K}$.

**Lema 2.** En un problema de clasificación multiclase, dado que $R(f^*) = 0$ y $0 \leq L_{\text{norm}}(f(x), k) \leq \frac{1}{K-1}$, $\forall k$, cualquier función de pérdida normalizada $L_{\text{norm}}$ es tolerante al ruido bajo ruido de etiqueta asimétrico (o condicional a la clase), si la tasa de ruido $\eta_{jk} < 1 - \eta_y$.

Denotamos el riesgo del clasificador $f$ bajo etiquetas limpias como $R(f) = \mathbb{E}_{x,y^*} [L_{\text{norm}}]$, y el riesgo bajo la tasa de ruido de etiqueta $\eta$ como $R_{\eta}(f) = \mathbb{E}_{x,y} [L_{\text{norm}}]$. Sean $f^*$ y $f^*_{\eta}$ los minimizadores globales de $R(f)$ y $R_{\eta}(f)$, respectivamente. Necesitamos probar que $f^*$ también es un minimizador global del riesgo ruidoso $R_{\eta}(f)$ para que $L$ sea robusta. Las condiciones de la tasa de ruido en el Lema 1 ($\eta < \frac{K-1}{K}$) y en el Lema 2 ($\eta_{jk} < 1 - \eta_y$) generalmente requieren que las etiquetas correctas sigan siendo la mayoría de la clase. En el Lema 2, la condición restrictiva $R(f^*) = 0$ puede no ser satisfecha en la práctica (por ejemplo, las clases pueden no ser completamente separables); sin embargo, aún se puede lograr una buena robustez empírica. Mientras que la condición $0 \leq L_{\text{norm}}(f(x), k) \leq \frac{1}{K-1}$, $\forall k$ puede ser fácilmente satisfecha por una función de pérdida típica.


## *Custom Loss (NCE)*

In [ ]:
class NormalizedCrossEntropy(tf.keras.losses.Loss):
    def __init__(self, num_classes, scale=1.0, name="normalized_cross_entropy"):
        super(NormalizedCrossEntropy, self).__init__(name=name)
        self.num_classes = num_classes
        self.scale = scale

    def call(self, y_true, y_pred):
        # Aplicar log_softmax a las predicciones
        # y_pred = tf.nn.log_softmax(y_pred, axis=1)
        y_true = tf.cast(y_true, tf.float32)

        # Convertir etiquetas verdaderas a one-hot
        # y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), self.num_classes)
        #y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

        # Calcular la entropía cruzada normalizada
        nce = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1) / (-tf.reduce_sum(tf.math.log(y_pred), axis=1))

        # Escalar y devolver la pérdida media
        # return self.scale * tf.reduce_mean(nce)
        return tf.reduce_mean(nce)

## *Models*

In [ ]:
def EEGNet(nb_classes, Chans = 64, Samples = 320,
             dropoutRate = 0.5, kernLength = 64, F1 = 8,
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:

        1. Depthwise Convolutions to learn spatial filters within a
        temporal convolution. The use of the depth_multiplier option maps
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn
        spatial filters within each filter in a filter-bank. This also limits
        the number of free parameters to fit when compared to a fully-connected
        convolution.

        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions.


    While the original paper used Dropout, we found that SpatialDropout2D
    sometimes produced slightly better results for classification of ERP
    signals. However, SpatialDropout2D significantly reduced performance
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.

    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the
    kernel lengths for double the sampling rate, etc). Note that we haven't
    tested the model performance with this rule so this may not work well.

    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D.
    Inputs:

      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D.
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """

    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   name='Conv2D_1',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   name='Depth_wise_Conv2D_1',
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    block2       = SeparableConv2D(F2, (1, 16),
                                   name='Separable_Conv2D_1',
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense        = Dense(nb_classes, name = 'output',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'out_activation')(dense)

    return Model(inputs=input1, outputs=softmax)

In [ ]:
class Sampling(Layer):
    def call(self, inputs):
        mean, log_var = inputs
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mean + tf.exp(0.5 * log_var) * epsilon

def EEGNetVAE(nb_classes, Chans = 64, Samples = 320,
             dropoutRate = 0.5, kernLength = 128, F1 = 8,
             D = 2, F2 = 16, norm_rate = 0.25, codings_size = 10, dropoutType = 'Dropout', om = 0.5):



    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:

        1. Depthwise Convolutions to learn spatial filters within a
        temporal convolution. The use of the depth_multiplier option maps
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn
        spatial filters within each filter in a filter-bank. This also limits
        the number of free parameters to fit when compared to a fully-connected
        convolution.

        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions.


    While the original paper used Dropout, we found that SpatialDropout2D
    sometimes produced slightly better results for classification of ERP
    signals. However, SpatialDropout2D significantly reduced performance
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.

    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the
    kernel lengths for double the sampling rate, etc). Note that we haven't
    tested the model performance with this rule so this may not work well.

    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D.
    Inputs:

      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D.
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """

    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   name='Conv2D_1',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)

    block1       = BatchNormalization()(block1)
    block1       = Conv2D(F1*D,(Chans, 1), use_bias = False, name = 'Depth_wise_remplace')(block1)
    #block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   #name='Depth_wise_Conv2D_1',
                                   #depth_multiplier = D,
                                   #depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    #block2       = SeparableConv2D(F2, (1, 16),
                                   #name='Separable_Conv2D_1',
                                   #use_bias = False, padding = 'same')(block1)
    block2       = Conv2D(F1*D, (1,32), padding = 'same', name = 'Separable_remplace_1', use_bias = False)(block1)
    block2       = Conv2D(F2, (1,1), name = 'Separable_remplace_2', use_bias = False)(block2)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
    flatten_1      = Flatten(name = 'flatten_1')(block2)
    
    codings_mean = keras.layers.Dense(codings_size, name='codings_mean_1')(flatten_1)
    codings_log_var = keras.layers.Dense(codings_size, name='codings_log_var_1')(flatten_1)
    codings = Sampling(name='sampling')([codings_mean, codings_log_var])


    # SeparableDecoder
    decoded = dropoutType(dropoutRate)(codings)
    decoded = Dense(block2.shape[1] * block2.shape[2] * block2.shape[3])(decoded)
    decoded = tf.reshape(decoded, [-1, block2.shape[1], block2.shape[2], block2.shape[3]])
    decoded = UpSampling2D(size= (1,8), interpolation= 'nearest', data_format= 'channels_last', name= 'AveragePooling2D_BE_Decoded')(decoded)
    decoded = Activation('elu', name= 'ActivationB2_Decoded')(decoded)
    decoded = BatchNormalization(name= 'BatchNormalizationB2_Decoded')(decoded)
    decoded = Conv2DTranspose(F2, (1,1), use_bias = False, name = 'SSeparable_remplace_2_decoded')(decoded)
    decoded = Conv2DTranspose(F1*D, (1,32), padding = 'same', name = 'Separable_remplace_1_decoded', use_bias = False)(decoded)

    # DepthwiseDecoder
    decoded = dropoutType(dropoutRate, name= 'DropoutTypeB1_Decoded')(decoded)
    decoded = UpSampling2D(size= (1,4), interpolation= 'nearest', data_format= 'channels_last', name= 'AveragePooling2D_B1_Decoded')(decoded)
    decoded = Activation('elu', name= 'ActivationB1_Decoded')(decoded)
    decoded = BatchNormalization(name= 'BatchNormalizationB1.2_Decoded')(decoded)
    decoded = Conv2DTranspose(F1*D , (Chans,1),
                 name='Depth_wise_remplace_decoded',
                 use_bias=False)(decoded)

    # Conv2DDecoder
    decoded = BatchNormalization(name= 'BatchNormalization1.1_Decoded')(decoded)
    decoded = Conv2D(1 , (1,kernLength), padding='same',
             name='Conv2D_1_Decoded',
             use_bias=False)(decoded)

    dense        = Dense(nb_classes, name = 'output',
                         kernel_constraint = max_norm(norm_rate))(flatten_1)
    softmax      = Activation('softmax', name = 'out_activation')(dense)

    model = Model(inputs=input1, outputs= softmax)
    
    KL = -0.5 * tf.keras.backend.sum( 1 + codings_log_var - tf.keras.backend.exp(codings_log_var) - tf.keras.backend.square(codings_mean),axis=-1)
    KL = 0.2*(tf.keras.backend.mean(KL)/codings_log_var.shape[-1])#Chans*Samples
    model.add_loss(KL)#Chans*Samples)


    return model

## *Load Dataset*

## *EEGNetVAE - Sbj: 43,14,6*

In [ ]:
sbj=43

X , y = load_GIGA(sbj=sbj, **load_args)

print("===============================")
print(X.shape)
print(y.shape)
print("===============================")
Acc_runs = []

X_train_run_0 , y_train_run_0 = load_GIGA(sbj=sbj,run=0,**load_args)
X_train_run_1 , y_train_run_1 = load_GIGA(sbj=sbj,run=1,**load_args)
X_train_run_2 , y_train_run_2 = load_GIGA(sbj=sbj,run=2,**load_args)
X_train_run_3 , y_train_run_3 = load_GIGA(sbj=sbj,run=3,**load_args)
X_train_run_4 , y_train_run_4 = load_GIGA(sbj=sbj,run=4,**load_args)

print("===============================")
print("Training X Run 0 data shape:", X_train_run_0.shape)
print(y_train_run_0.shape)
print("Training X Run 1 data shape:",X_train_run_1.shape)
print(y_train_run_1.shape)
print("Training X Run 2 data shape:",X_train_run_2.shape)
print(y_train_run_2.shape)
print("Training X Run 3 data shape:",X_train_run_3.shape)
print(y_train_run_3.shape)
print("Training X Run 4 data shape:",X_train_run_4.shape)
print(y_train_run_4.shape)
print("===============================")

list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]

## *Training*

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Función para graficar loss y accuracy por cada fold
def plot_metrics(history, run_idx, fold_idx):
    # Graficar Loss
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Lista para almacenar las métricas de cada iteración
Acc_runs43 = []
n_folds = 5

# Loop para la validación LORO, dejando un Run fuera para validación
for run_idx in range(len(list_train)):
    print(f"\n=== Validación LORO con Run {run_idx + 1} excluido ===")
    
    # Datos de test: el Run excluido
    X_test = list_train[run_idx]
    y_test = list_y_train[run_idx]
    
    # Datos de entrenamiento: todos los Runs excepto el excluido
    X_train = np.concatenate([list_train[i] for i in range(len(list_train)) if i != run_idx], axis=0)
    y_train = np.concatenate([list_y_train[i] for i in range(len(list_y_train)) if i != run_idx], axis=0)
    
    # Validación por Folds con shuffle
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        print(f"--- Fold {fold_idx + 1}/{n_folds} ---")
        
        # Separar los datos en entrenamiento y validación para el Fold actual
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Crear el modelo (ajusta create_model según tu arquitectura)
        model43 = EEGNetVAE(nb_classes=2)
        model43.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss={'out_activation': 'sparse_categorical_crossentropy'},
                      loss_weights={'out_activation': 0.8}, metrics=['accuracy'])
        
        # Entrenar el modelo en los datos del fold actual
        history43 = model43.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=200,  # Ajusta el número de épocas según tus necesidades
                            batch_size=16,
                            verbose=0)
        
        # Graficar las métricas de loss y accuracy
        plot_metrics(history43, run_idx, fold_idx)
        
        # Evaluar en el fold de validación
        val_loss, val_acc = model43.evaluate(X_fold_val, y_fold_val)
        print(f"Fold {fold_idx + 1} - Val Loss: {val_loss}, Val Acc: {val_acc}")
        
        # Evaluar en el Run de test (LORO)
        test_loss, test_acc = model43.evaluate(X_test, y_test)
        print(f"Run {run_idx + 1} - Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        # Guardar resultados
        Acc_runs43.append({
            'run': run_idx + 1,
            'fold': fold_idx + 1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'test_loss': test_loss,
            'test_acc': test_acc
        })

# Mostrar los resultados finales
print("\n=== Resultados Finales ===")
for result in Acc_runs43:
    print(result)


In [ ]:
sbj=14

X , y = load_GIGA(sbj=sbj, **load_args)

print("===============================")
print(X.shape)
print(y.shape)
print("===============================")
Acc_runs = []

X_train_run_0 , y_train_run_0 = load_GIGA(sbj=sbj,run=0,**load_args)
X_train_run_1 , y_train_run_1 = load_GIGA(sbj=sbj,run=1,**load_args)
X_train_run_2 , y_train_run_2 = load_GIGA(sbj=sbj,run=2,**load_args)
X_train_run_3 , y_train_run_3 = load_GIGA(sbj=sbj,run=3,**load_args)
X_train_run_4 , y_train_run_4 = load_GIGA(sbj=sbj,run=4,**load_args)

print("===============================")
print("Training X Run 0 data shape:", X_train_run_0.shape)
print(y_train_run_0.shape)
print("Training X Run 1 data shape:",X_train_run_1.shape)
print(y_train_run_1.shape)
print("Training X Run 2 data shape:",X_train_run_2.shape)
print(y_train_run_2.shape)
print("Training X Run 3 data shape:",X_train_run_3.shape)
print(y_train_run_3.shape)
print("Training X Run 4 data shape:",X_train_run_4.shape)
print(y_train_run_4.shape)
print("===============================")

list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Función para graficar loss y accuracy por cada fold
def plot_metrics(history, run_idx, fold_idx):
    # Graficar Loss
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Lista para almacenar las métricas de cada iteración
Acc_runs14 = []
n_folds = 5

# Loop para la validación LORO, dejando un Run fuera para validación
for run_idx in range(len(list_train)):
    print(f"\n=== Validación LORO con Run {run_idx + 1} excluido ===")
    
    # Datos de test: el Run excluido
    X_test = list_train[run_idx]
    y_test = list_y_train[run_idx]
    
    # Datos de entrenamiento: todos los Runs excepto el excluido
    X_train = np.concatenate([list_train[i] for i in range(len(list_train)) if i != run_idx], axis=0)
    y_train = np.concatenate([list_y_train[i] for i in range(len(list_y_train)) if i != run_idx], axis=0)
    
    # Validación por Folds con shuffle
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        print(f"--- Fold {fold_idx + 1}/{n_folds} ---")
        
        # Separar los datos en entrenamiento y validación para el Fold actual
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Crear el modelo (ajusta create_model según tu arquitectura)
        model14 = EEGNetVAE(nb_classes=2)
        model14.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss={'out_activation': 'sparse_categorical_crossentropy'},
                      loss_weights={'out_activation': 0.8}, metrics=['accuracy'])
        
        # Entrenar el modelo en los datos del fold actual
        history14 = model14.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=200,  # Ajusta el número de épocas según tus necesidades
                            batch_size=16,
                            verbose=0)
        
        # Graficar las métricas de loss y accuracy
        plot_metrics(history14, run_idx, fold_idx)
        
        # Evaluar en el fold de validación
        val_loss, val_acc = model14.evaluate(X_fold_val, y_fold_val)
        print(f"Fold {fold_idx + 1} - Val Loss: {val_loss}, Val Acc: {val_acc}")
        
        # Evaluar en el Run de test (LORO)
        test_loss, test_acc = model14.evaluate(X_test, y_test)
        print(f"Run {run_idx + 1} - Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        # Guardar resultados
        Acc_runs14.append({
            'run': run_idx + 1,
            'fold': fold_idx + 1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'test_loss': test_loss,
            'test_acc': test_acc
        })

# Mostrar los resultados finales
print("\n=== Resultados Finales ===")
for result in Acc_runs14:
    print(result)


In [ ]:
sbj=6

X , y = load_GIGA(sbj=sbj, **load_args)

print("===============================")
print(X.shape)
print(y.shape)
print("===============================")
Acc_runs = []

X_train_run_0 , y_train_run_0 = load_GIGA(sbj=sbj,run=0,**load_args)
X_train_run_1 , y_train_run_1 = load_GIGA(sbj=sbj,run=1,**load_args)
X_train_run_2 , y_train_run_2 = load_GIGA(sbj=sbj,run=2,**load_args)
X_train_run_3 , y_train_run_3 = load_GIGA(sbj=sbj,run=3,**load_args)
X_train_run_4 , y_train_run_4 = load_GIGA(sbj=sbj,run=4,**load_args)

print("===============================")
print("Training X Run 0 data shape:", X_train_run_0.shape)
print(y_train_run_0.shape)
print("Training X Run 1 data shape:",X_train_run_1.shape)
print(y_train_run_1.shape)
print("Training X Run 2 data shape:",X_train_run_2.shape)
print(y_train_run_2.shape)
print("Training X Run 3 data shape:",X_train_run_3.shape)
print(y_train_run_3.shape)
print("Training X Run 4 data shape:",X_train_run_4.shape)
print(y_train_run_4.shape)
print("===============================")

list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]

In [ ]:
n_folds = 5

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Función para graficar loss y accuracy por cada fold
def plot_metrics(history, run_idx, fold_idx):
    # Graficar Loss
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Lista para almacenar las métricas de cada iteración
Acc_runs6 = []
n_folds = 5

# Loop para la validación LORO, dejando un Run fuera para validación
for run_idx in range(len(list_train)):
    print(f"\n=== Validación LORO con Run {run_idx + 1} excluido ===")
    
    # Datos de test: el Run excluido
    X_test = list_train[run_idx]
    y_test = list_y_train[run_idx]
    
    # Datos de entrenamiento: todos los Runs excepto el excluido
    X_train = np.concatenate([list_train[i] for i in range(len(list_train)) if i != run_idx], axis=0)
    y_train = np.concatenate([list_y_train[i] for i in range(len(list_y_train)) if i != run_idx], axis=0)
    
    # Validación por Folds con shuffle
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        print(f"--- Fold {fold_idx + 1}/{n_folds} ---")
        
        # Separar los datos en entrenamiento y validación para el Fold actual
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Crear el modelo (ajusta create_model según tu arquitectura)
        model6 = EEGNetVAE(nb_classes=2)
        model6.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss={'out_activation': 'sparse_categorical_crossentropy'},
                      loss_weights={'out_activation': 0.8}, metrics=['accuracy'])
        
        # Entrenar el modelo en los datos del fold actual
        history6 = model6.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=200,  # Ajusta el número de épocas según tus necesidades
                            batch_size=16,
                            verbose=0)
        
        # Graficar las métricas de loss y accuracy
        plot_metrics(history6, run_idx, fold_idx)
        
        # Evaluar en el fold de validación
        val_loss, val_acc = model6.evaluate(X_fold_val, y_fold_val)
        print(f"Fold {fold_idx + 1} - Val Loss: {val_loss}, Val Acc: {val_acc}")
        
        # Evaluar en el Run de test (LORO)
        test_loss, test_acc = model6.evaluate(X_test, y_test)
        print(f"Run {run_idx + 1} - Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        # Guardar resultados
        Acc_runs6.append({
            'run': run_idx + 1,
            'fold': fold_idx + 1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'test_loss': test_loss,
            'test_acc': test_acc
        })

# Mostrar los resultados finales
print("\n=== Resultados Finales ===")
for result in Acc_runs6:
    print(result)


## *EEGNet - sbj: 43,14,6*

In [ ]:
sbj=43

X , y = load_GIGA(sbj=sbj, **load_args)

print("===============================")
print(X.shape)
print(y.shape)
print("===============================")
Acc_runs = []

X_train_run_0 , y_train_run_0 = load_GIGA(sbj=sbj,run=0,**load_args)
X_train_run_1 , y_train_run_1 = load_GIGA(sbj=sbj,run=1,**load_args)
X_train_run_2 , y_train_run_2 = load_GIGA(sbj=sbj,run=2,**load_args)
X_train_run_3 , y_train_run_3 = load_GIGA(sbj=sbj,run=3,**load_args)
X_train_run_4 , y_train_run_4 = load_GIGA(sbj=sbj,run=4,**load_args)

print("===============================")
print("Training X Run 0 data shape:", X_train_run_0.shape)
print(y_train_run_0.shape)
print("Training X Run 1 data shape:",X_train_run_1.shape)
print(y_train_run_1.shape)
print("Training X Run 2 data shape:",X_train_run_2.shape)
print(y_train_run_2.shape)
print("Training X Run 3 data shape:",X_train_run_3.shape)
print(y_train_run_3.shape)
print("Training X Run 4 data shape:",X_train_run_4.shape)
print(y_train_run_4.shape)
print("===============================")

list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Función para graficar loss y accuracy por cada fold
def plot_metrics(history, run_idx, fold_idx):
    # Graficar Loss
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Lista para almacenar las métricas de cada iteración
Acc_runs43 = []
n_folds = 5

# Loop para la validación LORO, dejando un Run fuera para validación
for run_idx in range(len(list_train)):
    print(f"\n=== Validación LORO con Run {run_idx + 1} excluido ===")
    
    # Datos de test: el Run excluido
    X_test = list_train[run_idx]
    y_test = list_y_train[run_idx]
    
    # Datos de entrenamiento: todos los Runs excepto el excluido
    X_train = np.concatenate([list_train[i] for i in range(len(list_train)) if i != run_idx], axis=0)
    y_train = np.concatenate([list_y_train[i] for i in range(len(list_y_train)) if i != run_idx], axis=0)
    
    # Validación por Folds con shuffle
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        print(f"--- Fold {fold_idx + 1}/{n_folds} ---")
        
        # Separar los datos en entrenamiento y validación para el Fold actual
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Crear el modelo (ajusta create_model según tu arquitectura)
        model43 = EEGNet(nb_classes=2)
        model43.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss={'out_activation': 'sparse_categorical_crossentropy'},
                      loss_weights={'out_activation': 0.8}, metrics=['accuracy'])
        
        # Entrenar el modelo en los datos del fold actual
        history43 = model43.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=200,  # Ajusta el número de épocas según tus necesidades
                            batch_size=16,
                            verbose=0)
        
        # Graficar las métricas de loss y accuracy
        plot_metrics(history43, run_idx, fold_idx)
        
        # Evaluar en el fold de validación
        val_loss, val_acc = model43.evaluate(X_fold_val, y_fold_val)
        print(f"Fold {fold_idx + 1} - Val Loss: {val_loss}, Val Acc: {val_acc}")
        
        # Evaluar en el Run de test (LORO)
        test_loss, test_acc = model43.evaluate(X_test, y_test)
        print(f"Run {run_idx + 1} - Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        # Guardar resultados
        Acc_runs43.append({
            'run': run_idx + 1,
            'fold': fold_idx + 1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'test_loss': test_loss,
            'test_acc': test_acc
        })

# Mostrar los resultados finales
print("\n=== Resultados Finales ===")
for result in Acc_runs43:
    print(result)


In [ ]:
sbj=14

X , y = load_GIGA(sbj=sbj, **load_args)

print("===============================")
print(X.shape)
print(y.shape)
print("===============================")
Acc_runs = []

X_train_run_0 , y_train_run_0 = load_GIGA(sbj=sbj,run=0,**load_args)
X_train_run_1 , y_train_run_1 = load_GIGA(sbj=sbj,run=1,**load_args)
X_train_run_2 , y_train_run_2 = load_GIGA(sbj=sbj,run=2,**load_args)
X_train_run_3 , y_train_run_3 = load_GIGA(sbj=sbj,run=3,**load_args)
X_train_run_4 , y_train_run_4 = load_GIGA(sbj=sbj,run=4,**load_args)

print("===============================")
print("Training X Run 0 data shape:", X_train_run_0.shape)
print(y_train_run_0.shape)
print("Training X Run 1 data shape:",X_train_run_1.shape)
print(y_train_run_1.shape)
print("Training X Run 2 data shape:",X_train_run_2.shape)
print(y_train_run_2.shape)
print("Training X Run 3 data shape:",X_train_run_3.shape)
print(y_train_run_3.shape)
print("Training X Run 4 data shape:",X_train_run_4.shape)
print(y_train_run_4.shape)
print("===============================")

list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Función para graficar loss y accuracy por cada fold
def plot_metrics(history, run_idx, fold_idx):
    # Graficar Loss
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Lista para almacenar las métricas de cada iteración
Acc_runs14 = []
n_folds = 5

# Loop para la validación LORO, dejando un Run fuera para validación
for run_idx in range(len(list_train)):
    print(f"\n=== Validación LORO con Run {run_idx + 1} excluido ===")
    
    # Datos de test: el Run excluido
    X_test = list_train[run_idx]
    y_test = list_y_train[run_idx]
    
    # Datos de entrenamiento: todos los Runs excepto el excluido
    X_train = np.concatenate([list_train[i] for i in range(len(list_train)) if i != run_idx], axis=0)
    y_train = np.concatenate([list_y_train[i] for i in range(len(list_y_train)) if i != run_idx], axis=0)
    
    # Validación por Folds con shuffle
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        print(f"--- Fold {fold_idx + 1}/{n_folds} ---")
        
        # Separar los datos en entrenamiento y validación para el Fold actual
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Crear el modelo (ajusta create_model según tu arquitectura)
        model14 = EEGNet(nb_classes=2)
        model14.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss={'out_activation': 'sparse_categorical_crossentropy'},
                      loss_weights={'out_activation': 0.8}, metrics=['accuracy'])
        
        # Entrenar el modelo en los datos del fold actual
        history14 = model14.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=200,  # Ajusta el número de épocas según tus necesidades
                            batch_size=16,
                            verbose=0)
        
        # Graficar las métricas de loss y accuracy
        plot_metrics(history14, run_idx, fold_idx)
        
        # Evaluar en el fold de validación
        val_loss, val_acc = model14.evaluate(X_fold_val, y_fold_val)
        print(f"Fold {fold_idx + 1} - Val Loss: {val_loss}, Val Acc: {val_acc}")
        
        # Evaluar en el Run de test (LORO)
        test_loss, test_acc = model14.evaluate(X_test, y_test)
        print(f"Run {run_idx + 1} - Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        # Guardar resultados
        Acc_runs14.append({
            'run': run_idx + 1,
            'fold': fold_idx + 1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'test_loss': test_loss,
            'test_acc': test_acc
        })

# Mostrar los resultados finales
print("\n=== Resultados Finales ===")
for result in Acc_runs14:
    print(result)


In [ ]:
sbj=6

X , y = load_GIGA(sbj=sbj, **load_args)

print("===============================")
print(X.shape)
print(y.shape)
print("===============================")
Acc_runs = []

X_train_run_0 , y_train_run_0 = load_GIGA(sbj=sbj,run=0,**load_args)
X_train_run_1 , y_train_run_1 = load_GIGA(sbj=sbj,run=1,**load_args)
X_train_run_2 , y_train_run_2 = load_GIGA(sbj=sbj,run=2,**load_args)
X_train_run_3 , y_train_run_3 = load_GIGA(sbj=sbj,run=3,**load_args)
X_train_run_4 , y_train_run_4 = load_GIGA(sbj=sbj,run=4,**load_args)

print("===============================")
print("Training X Run 0 data shape:", X_train_run_0.shape)
print(y_train_run_0.shape)
print("Training X Run 1 data shape:",X_train_run_1.shape)
print(y_train_run_1.shape)
print("Training X Run 2 data shape:",X_train_run_2.shape)
print(y_train_run_2.shape)
print("Training X Run 3 data shape:",X_train_run_3.shape)
print(y_train_run_3.shape)
print("Training X Run 4 data shape:",X_train_run_4.shape)
print(y_train_run_4.shape)
print("===============================")

list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Función para graficar loss y accuracy por cada fold
def plot_metrics(history, run_idx, fold_idx):
    # Graficar Loss
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Run {run_idx + 1} - Fold {fold_idx + 1} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Lista para almacenar las métricas de cada iteración
Acc_runs6 = []
n_folds = 5

# Loop para la validación LORO, dejando un Run fuera para validación
for run_idx in range(len(list_train)):
    print(f"\n=== Validación LORO con Run {run_idx + 1} excluido ===")
    
    # Datos de test: el Run excluido
    X_test = list_train[run_idx]
    y_test = list_y_train[run_idx]
    
    # Datos de entrenamiento: todos los Runs excepto el excluido
    X_train = np.concatenate([list_train[i] for i in range(len(list_train)) if i != run_idx], axis=0)
    y_train = np.concatenate([list_y_train[i] for i in range(len(list_y_train)) if i != run_idx], axis=0)
    
    # Validación por Folds con shuffle
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        print(f"--- Fold {fold_idx + 1}/{n_folds} ---")
        
        # Separar los datos en entrenamiento y validación para el Fold actual
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Crear el modelo (ajusta create_model según tu arquitectura)
        model6 = EEGNet(nb_classes=2)
        model6.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss={'out_activation': 'sparse_categorical_crossentropy'},
                      loss_weights={'out_activation': 0.8}, metrics=['accuracy'])
        
        # Entrenar el modelo en los datos del fold actual
        history6 = model6.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=200,  # Ajusta el número de épocas según tus necesidades
                            batch_size=16,
                            verbose=0)
        
        # Graficar las métricas de loss y accuracy
        plot_metrics(history6, run_idx, fold_idx)
        
        # Evaluar en el fold de validación
        val_loss, val_acc = model6.evaluate(X_fold_val, y_fold_val)
        print(f"Fold {fold_idx + 1} - Val Loss: {val_loss}, Val Acc: {val_acc}")
        
        # Evaluar en el Run de test (LORO)
        test_loss, test_acc = model6.evaluate(X_test, y_test)
        print(f"Run {run_idx + 1} - Test Loss: {test_loss}, Test Acc: {test_acc}")
        
        # Guardar resultados
        Acc_runs6.append({
            'run': run_idx + 1,
            'fold': fold_idx + 1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'test_loss': test_loss,
            'test_acc': test_acc
        })

# Mostrar los resultados finales
print("\n=== Resultados Finales ===")
for result in Acc_runs6:
    print(result)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt


# ----------------------------------------------------------------------
def agco(method_1, method_2, ticks, labels, sort='method_2', reference_c='C1', gain_c='C0', loss_c='C3', barwidth=6, ylabel='Accuracy [%]', xlabel='Subjects', gain_labels=['gain', 'loss'], fig=None, ax=None, size=(15, 5), dpi=90, **kwargs):
    """"""
    if 'lose_c' in kwargs:
        loss_c = kwargs['lose_c']

    if fig is None:
        plt.figure(figsize=size, dpi=dpi)

    if ax is None:
        ax = plt.subplot(111)


    if sort is None:
        index = np.arange(len(method_2))
    elif sort == 'method_1':
        index = np.argsort(method_1)[::-1]
    elif sort == 'method_1r':
        index = np.argsort(method_1)
    elif sort == 'method_2':
        index = np.argsort(method_2)[::-1]
    elif sort == 'method_2r':
        index = np.argsort(method_2)
    else:
        index = sort

    colors = np.array(method_2[index]
                      - method_1[index] < 0, dtype=np.object_)

    index = range(len(method_1))
    
    if sort=='method_2':
        p1, = plt.plot(method_2[index], color=reference_c, linestyle='--',)
        p2, = plt.plot(method_1[index], color=gain_c,
                       linestyle='--', alpha=0.3)
        colors[colors == 0] = loss_c
        colors[colors == 1] = gain_c
    else:
        p1, = plt.plot(method_1[index], color=reference_c, linestyle='--',)
        p2, = plt.plot(method_2[index], color=gain_c,
                       linestyle='--', alpha=0.3)
        colors[colors == 0] = gain_c
        colors[colors == 1] = loss_c

    plots = [p1, p2]

    if np.array([colors == gain_c]).any():
        p3 = plt.vlines(np.array(sorted(index))[colors == gain_c], method_1[index][colors == gain_c],
                        method_2[index][colors == gain_c], color=colors[colors == gain_c], linewidth=barwidth)
        labels.append(gain_labels[0])
        plots.append(p3)

    if np.array([colors == loss_c]).any():
        p4 = plt.vlines(np.array(sorted(index))[colors == loss_c], method_1[index][colors == loss_c],
                        method_2[index][colors == loss_c], color=colors[colors == loss_c], linewidth=barwidth)
        labels.append(gain_labels[1])
        plots.append(p4)

    plt.ylabel(ylabel, fontsize=15)
    plt.xlabel(xlabel, fontsize=15)
    plt.xticks(range(len(ticks)), ticks[index], rotation=90)

    try:
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
    except:
        pass

    try:
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
    except:
        pass

    l1 = plt.legend(plots, labels, loc='upper center',
                    ncol=2, bbox_to_anchor=(0.5, 1), fontsize=12)
    plt.gca().add_artist(l1)

    return plt.gcf()

In [ ]:
import numpy as np
ticks_T2 = np.array(['R1F1', 'R1F2', 'R1F3', 'R1F4', 'R1F5', 'R2F1', 'R2F2', 'R2F3', 'R2F4', 'R2F5', 'R3F1', 'R3F2', 'R3F3', 'R3F4', 'R3F5', 'R4F1', 'R4F2', 'R4F3', 'R4F4', 'R4F5', 'R5F1' ,'R5F2' ,'R5F3', 'R5F4', 'R5F5'])

method_1T2 = np.array([
    0.574999988079071,  # Run 1 Fold 1
    0.5,                # Run 1 Fold 2
    0.824999988079071,  # Run 1 Fold 3
    0.5,                # Run 1 Fold 4
    0.675000011920929,  # Run 1 Fold 5
    0.925000011920929,  # Run 2 Fold 1
    0.8999999761581421, # Run 2 Fold 2
    0.75,               # Run 2 Fold 3
    0.9750000238418579, # Run 2 Fold 4
    1.0,                # Run 2 Fold 5
    1.0,                # Run 3 Fold 1
    0.8500000238418579, # Run 3 Fold 2
    1.0,                # Run 3 Fold 3
    0.8500000238418579, # Run 3 Fold 4
    0.925000011920929,  # Run 3 Fold 5
    0.9750000238418579, # Run 4 Fold 1
    1.0,                # Run 4 Fold 2
    0.9750000238418579, # Run 4 Fold 3
    1.0,                # Run 4 Fold 4
    0.949999988079071,  # Run 4 Fold 5
    0.9750000238418579, # Run 5 Fold 1
    0.875,              # Run 5 Fold 2
    0.949999988079071,  # Run 5 Fold 3
    0.824999988079071,  # Run 5 Fold 4
    1.0                 # Run 5 Fold 5
])*100

method_2T2 = np.array([
    0.6499999761581421,  # run 1, fold 1
    0.675000011920929,   # run 1, fold 2
    0.7250000238418579,  # run 1, fold 3
    0.6499999761581421,  # run 1, fold 4
    0.574999988079071,   # run 1, fold 5
    0.9750000238418579,  # run 2, fold 1
    0.9750000238418579,  # run 2, fold 2
    0.9750000238418579,  # run 2, fold 3
    0.8500000238418579,  # run 2, fold 4
    0.8500000238418579,  # run 2, fold 5
    1.0,                 # run 3, fold 1
    1.0,                 # run 3, fold 2
    1.0,                 # run 3, fold 3
    0.9750000238418579,  # run 3, fold 4
    1.0,                 # run 3, fold 5
    0.949999988079071,   # run 4, fold 1
    0.9750000238418579,  # run 4, fold 2
    1.0,                 # run 4, fold 3
    0.949999988079071,   # run 4, fold 4
    1.0,                 # run 4, fold 5
    0.949999988079071,   # run 5, fold 1
    0.9750000238418579,  # run 5, fold 2
    0.925000011920929,   # run 5, fold 3
    0.8999999761581421,  # run 5, fold 4
    0.8500000238418579   # run 5, fold 5
])*100


ticks_T3 = np.array(['R1F1' 'R1F2' 'R1F3' 'R1F4' 'R1F5' 'R2F1' 'R2F2' 'R2F3' 'R2F4' 'R2F5' 'R3F1' 'R3F2' 'R3F3' 'R3F4' 'R3F5' 'R4F1' 'R4F2' 'R4F3' 'R4F4' 'R4F5' 'R5F1' 'R5F2' 'R5F3' 'R5F4' 'R5F5'])

method_1T3 = np.array([
    0.9750000238418579,  # Run 1 Fold 1
    0.9750000238418579,  # Run 1 Fold 2
    0.9750000238418579,  # Run 1 Fold 3
    0.9750000238418579,  # Run 1 Fold 4
    0.9750000238418579,  # Run 1 Fold 5
    0.8750000000000000,  # Run 2 Fold 1
    0.8500000238418579,  # Run 2 Fold 2
    0.8750000000000000,  # Run 2 Fold 3
    0.8750000000000000,  # Run 2 Fold 4
    0.949999988079071,   # Run 2 Fold 5
    0.9750000238418579,  # Run 3 Fold 1
    0.9750000238418579,  # Run 3 Fold 2
    0.9750000238418579,  # Run 3 Fold 3
    0.925000011920929,   # Run 3 Fold 4
    0.949999988079071,   # Run 3 Fold 5
    0.925000011920929,   # Run 4 Fold 1
    0.8750000000000000,  # Run 4 Fold 2
    0.8999999761581421,  # Run 4 Fold 3
    0.8999999761581421,  # Run 4 Fold 4
    0.8750000000000000,  # Run 4 Fold 5
    0.925000011920929,   # Run 5 Fold 1
    0.949999988079071,   # Run 5 Fold 2
    0.925000011920929,   # Run 5 Fold 3
    0.8999999761581421,  # Run 5 Fold 4
    0.949999988079071    # Run 5 Fold 5
])*100
method_2T3 = np.array([
    1.0,                # run 1, fold 1
    1.0,                # run 1, fold 2
    1.0,                # run 1, fold 3
    1.0,                # run 1, fold 4
    1.0,                # run 1, fold 5
    0.949999988079071, # run 2, fold 1
    0.949999988079071, # run 2, fold 2
    0.8999999761581421, # run 2, fold 3
    1.0,                # run 2, fold 4
    0.9750000238418579, # run 2, fold 5
    0.9750000238418579, # run 3, fold 1
    0.9750000238418579, # run 3, fold 2
    0.9750000238418579, # run 3, fold 3
    1.0,                # run 3, fold 4
    1.0,                # run 3, fold 5
    0.925000011920929, # run 4, fold 1
    0.925000011920929, # run 4, fold 2
    0.925000011920929, # run 4, fold 3
    0.925000011920929, # run 4, fold 4
    0.925000011920929, # run 4, fold 5
    0.8500000238418579, # run 5, fold 1
    0.8999999761581421, # run 5, fold 2
    0.875,             # run 5, fold 3
    0.8999999761581421, # run 5, fold 4
    0.925000011920929  # run 5, fold 5
])*100

ticks_T4 = np.array(['R1F1' 'R1F2' 'R1F3' 'R1F4' 'R1F5' 'R2F1' 'R2F2' 'R2F3' 'R2F4' 'R2F5' 'R3F1' 'R3F2' 'R3F3' 'R3F4' 'R3F5' 'R4F1' 'R4F2' 'R4F3' 'R4F4' 'R4F5' 'R5F1' 'R5F2' 'R5F3' 'R5F4' 'R5F5'])


method_1T4 = np.array([
    0.6111111044883728,  # Run 1 Fold 1
    0.4444444477558136,  # Run 1 Fold 2
    0.5555555820465088,  # Run 1 Fold 3
    0.4166666567325592,  # Run 1 Fold 4
    0.4722222089767456,  # Run 1 Fold 5
    0.8055555820465088,  # Run 2 Fold 1
    0.8888888955116272,  # Run 2 Fold 2
    0.9444444179534912,  # Run 2 Fold 3
    0.5833333134651184,  # Run 2 Fold 4
    0.8611111044883728,  # Run 2 Fold 5
    0.9411764740943909,  # Run 3 Fold 1
    0.970588207244873,   # Run 3 Fold 2
    0.970588207244873,   # Run 3 Fold 3
    0.9411764740943909,  # Run 3 Fold 4
    0.970588207244873,   # Run 3 Fold 5
    0.7222222089767456,  # Run 4 Fold 1
    1.0,                 # Run 4 Fold 2
    1.0,                 # Run 4 Fold 3
    0.9722222089767456,  # Run 4 Fold 4
    0.8333333134651184,  # Run 4 Fold 5
    0.5277777910232544,  # Run 5 Fold 1
    0.5,                 # Run 5 Fold 2
    0.6111111044883728,  # Run 5 Fold 3
    0.4166666567325592,  # Run 5 Fold 4
    0.5833333134651184   # Run 5 Fold 5
])*100
method_2T4 = np.array([
    0.4444444477558136,  # Run 1 Fold 1
    0.4444444477558136,  # Run 1 Fold 2
    0.5277777910232544,  # Run 1 Fold 3
    0.3055555522441864,  # Run 1 Fold 4
    0.5,                 # Run 1 Fold 5
    0.9444444179534912,  # Run 2 Fold 1
    0.9444444179534912,  # Run 2 Fold 2
    0.9166666865348816,  # Run 2 Fold 3
    0.9444444179534912,  # Run 2 Fold 4
    0.8888888955116272,  # Run 2 Fold 5
    0.970588207244873,    # Run 3 Fold 1
    0.970588207244873,    # Run 3 Fold 2
    0.7647058963775635,  # Run 3 Fold 3
    0.970588207244873,    # Run 3 Fold 4
    0.970588207244873,    # Run 3 Fold 5
    0.9722222089767456,  # Run 4 Fold 1
    0.9444444179534912,  # Run 4 Fold 2
    0.9444444179534912,  # Run 4 Fold 3
    0.9722222089767456,  # Run 4 Fold 4
    0.9444444179534912,  # Run 4 Fold 5
    0.5,                 # Run 5 Fold 1
    0.6666666865348816,  # Run 5 Fold 2
    0.5833333134651184,  # Run 5 Fold 3
    0.6388888955116272,  # Run 5 Fold 4
    0.6388888955116272   # Run 5 Fold 5
])*100

In [ ]:
len(ticks_T2)

In [ ]:
fig = agco(method_1T2, method_2T2, ticks_T2, labels= ['EEGNet', 'VAEEEGNet'], size=(25, 10), gain_labels=['loss', 'gain'])

pastel2 = plt.get_cmap('Pastel2')
#f3 = plt.fill_betweenx([35, 100], 39.5, 50, color=pastel2(1), alpha=0.8, label='hola')
#f2 = plt.fill_betweenx([35, 100], 16.5, 39.5, color=pastel2(5), alpha=0.8)
#f1 = plt.fill_betweenx([35, 100], -1, 16.5, color=pastel2(0), alpha=0.8)
#l2 = plt.legend([f1, f2, f3], ["Group 1", 'Group 2', 'Group 3'], loc=4, ncol=1, bbox_to_anchor =(1.07, 0.5), fontsize=12)

plt.savefig('accuracies.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = agco(method_1T3, method_2T3, ticks_T2, labels= ['EEGNet', 'VAEEEGNet'], size=(25, 10), gain_labels=['loss', 'gain'])

pastel2 = plt.get_cmap('Pastel2')
#f3 = plt.fill_betweenx([35, 100], 39.5, 50, color=pastel2(1), alpha=0.8, label='hola')
#f2 = plt.fill_betweenx([35, 100], 16.5, 39.5, color=pastel2(5), alpha=0.8)
#f1 = plt.fill_betweenx([35, 100], -1, 16.5, color=pastel2(0), alpha=0.8)
#l2 = plt.legend([f1, f2, f3], ["Group 1", 'Group 2', 'Group 3'], loc=4, ncol=1, bbox_to_anchor =(1.07, 0.5), fontsize=12)

plt.savefig('accuracies.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = agco(method_1T4, method_2T4, ticks_T2, labels= ['EEGNet', 'VAEEEGNet'], size=(25, 10), gain_labels=['loss', 'gain'])

pastel2 = plt.get_cmap('Pastel2')
#f3 = plt.fill_betweenx([35, 100], 39.5, 50, color=pastel2(1), alpha=0.8, label='hola')
#f2 = plt.fill_betweenx([35, 100], 16.5, 39.5, color=pastel2(5), alpha=0.8)
#f1 = plt.fill_betweenx([35, 100], -1, 16.5, color=pastel2(0), alpha=0.8)
#l2 = plt.legend([f1, f2, f3], ["Group 1", 'Group 2', 'Group 3'], loc=4, ncol=1, bbox_to_anchor =(1.07, 0.5), fontsize=12)

plt.savefig('accuracies.pdf', bbox_inches='tight')
plt.show()